In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import math as m 
dataset, meta = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

train_data, test_data = dataset['train'], dataset['test']

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

num_train_examples = meta.splits['train'].num_examples
num_test_examples = meta.splits['test'].num_examples

print("Train: {} \n Test: {}".format(num_train_examples, num_test_examples))

def normalize(data, labels):
    data=tf.cast(data, tf.float32)
    data /= 255
    return data, labels

train_data, test_data = train_data.map(normalize), test_data.map(normalize)
train_data, test_data = train_data.cache(), test_data.cache()

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu',
                            input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D((2,2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2,2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])


batch=42
train_data = train_data.cache().repeat().shuffle(num_train_examples).batch(batch)
test_data = test_data.cache().repeat().batch(batch)

model.fit(train_data, epochs=10, steps_per_epoch=m.ceil(num_train_examples/batch), verbose=2)

loss, test_acc = model.evaluate(test_data, steps=m.ceil(num_test_examples/32), verbose=2)

KeyboardInterrupt: 